In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

import os
import tensorflow as tf
print(tf.__version__)

import sys
import keras
import keras.backend as K
import json
np.random.seed(42)

 

2.3.0


In [ ]:
conv_df = pd.read_csv("./drive/My Drive/data/movie_dialogs_dataset/movie_conversations.txt", sep=r" \+\+\+\$\+\+\+ ", header=None)
conv_df.columns = ["pers1", "pers2", "movie", "replies"]

print(conv_df.head())
len(conv_df)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


  pers1 pers2 movie                           replies
0    u0    u2    m0  ['L194', 'L195', 'L196', 'L197']
1    u0    u2    m0                  ['L198', 'L199']
2    u0    u2    m0  ['L200', 'L201', 'L202', 'L203']
3    u0    u2    m0          ['L204', 'L205', 'L206']
4    u0    u2    m0                  ['L207', 'L208']


83097

In [ ]:
lines_df = pd.read_csv("./drive/My Drive/data/movie_dialogs_dataset/movie_lines.txt", sep=r" \+\+\+\$\+\+\+ ", header=None)
lines_df.columns = ["lineid", "pers", "movie", "pers_name", "line"]
lines_df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,lineid,pers,movie,pers_name,line
0,L1045,u0,m0,BIANCA,They do not!
1,L1044,u2,m0,CAMERON,They do to!
2,L985,u0,m0,BIANCA,I hope so.
3,L984,u2,m0,CAMERON,She okay?
4,L925,u0,m0,BIANCA,Let's go.


In [ ]:
QA_df = pd.DataFrame(columns=["q", "a"])
for i, row in conv_df.iterrows():
    arr = eval(row["replies"])
    pers1 = row["pers1"]
    pers2 = row["pers2"]
    nrow = {'q': None, 'a': None}
    for lineid in arr:
        line = lines_df.loc[lines_df['lineid'] == lineid].values[0]        
        if line is not None:
            if line[1] == pers1:
                nrow['q'] = line[4]
            else:
                nrow['a'] = line[4]
            if nrow['q'] is not None and nrow['a'] is not None:
                QA_df = QA_df.append(nrow, ignore_index=True)
                nrow = {'q': None, 'a': None}
                if len(QA_df)%100 == 0:
                    print("Writing ", len(QA_df), " qa pairs...")
print(len(QA_df))


Writing  100  qa pairs...
Writing  200  qa pairs...
Writing  300  qa pairs...
Writing  400  qa pairs...
Writing  500  qa pairs...
Writing  600  qa pairs...
Writing  700  qa pairs...
Writing  800  qa pairs...
Writing  900  qa pairs...
Writing  1000  qa pairs...
Writing  1100  qa pairs...
Writing  1200  qa pairs...
Writing  1300  qa pairs...
Writing  1400  qa pairs...
Writing  1500  qa pairs...
Writing  1600  qa pairs...
Writing  1700  qa pairs...
Writing  1800  qa pairs...
Writing  1900  qa pairs...
Writing  2000  qa pairs...
Writing  2100  qa pairs...
Writing  2200  qa pairs...
Writing  2300  qa pairs...
Writing  2400  qa pairs...
Writing  2500  qa pairs...
Writing  2600  qa pairs...
Writing  2700  qa pairs...
Writing  2800  qa pairs...
Writing  2900  qa pairs...
Writing  3000  qa pairs...
Writing  3100  qa pairs...
Writing  3200  qa pairs...
Writing  3300  qa pairs...
Writing  3400  qa pairs...
Writing  3500  qa pairs...
Writing  3600  qa pairs...
Writing  3700  qa pairs...
Writing  3

TypeError: ignored

In [ ]:
QA_df.to_csv("./drive/My Drive/data/movie_dialogs_dataset/QA.txt", sep=r"©")

In [ ]:
#Загрузка данных, создание словаря
import nltk
nltk.download("punkt")
nltk.download("stopwords")
from nltk.tokenize import RegexpTokenizer
from collections import Counter
from nltk.corpus import stopwords

tokenizer = RegexpTokenizer(r'\w+')

PAD = "<PAD>"
START = "<START>"
END = "<END>"
UNK = "<UNK>"

stopwords = stopwords.words('english')
print(stopwords)

def tokenize(line):
    return [w for w in tokenizer.tokenize(line.lower()) if w != '']

def generate_vocabs(texts):
    counter = Counter()
    for text in texts:
        counter.update(tokenize(text))
    v = [PAD, START, END, UNK] + [x for x, freq in counter.most_common(25000-4)] #отсекаем слишком редкие слова
    vocab = {token: index for index, token in enumerate(v)}
    vocab_inverse = {idx: w for w, idx in vocab.items()}
    return vocab, vocab_inverse

Q = QA_df['q'].values
A = QA_df['a'].values

vocab, vocab_inverse = generate_vocabs(Q+A)

len(vocab), vocab_inverse[0], vocab_inverse[1]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over',

(25000, '<PAD>', '<START>')

In [ ]:
#Преобразуем X сразу в токенизацию, с учетом словаря, заменяем сразу выброшенные слова на <UNK>

def prepare_X_vocab(xt, vocab=vocab, use_start=True):
    result = []
    for x in xt:
        tokenized = tokenize(x)
        t = [x if x in vocab else UNK for x in tokenized] + [END]
        if use_start:
            t = [START] + t
        result.append(t)
    return result

Qt = prepare_X_vocab(Q)
At = prepare_X_vocab(A, use_start=False)
Qt[:3]

[['<START>',
  'can',
  'we',
  'make',
  'this',
  'quick',
  '<UNK>',
  '<UNK>',
  'and',
  'andrew',
  'barrett',
  'are',
  'having',
  'an',
  'incredibly',
  'horrendous',
  'public',
  'break',
  'up',
  'on',
  'the',
  'quad',
  'again',
  '<END>'],
 ['<START>',
  'not',
  'the',
  'hacking',
  'and',
  'gagging',
  'and',
  'spitting',
  'part',
  'please',
  '<END>'],
 ['<START>',
  'you',
  're',
  'asking',
  'me',
  'out',
  'that',
  's',
  'so',
  'cute',
  'what',
  's',
  'your',
  'name',
  'again',
  '<END>']]

In [ ]:
EMBEDDING_DIM = 64
import keras
import keras.layers as L
from gensim.models import Word2Vec

w2vec = Word2Vec(Qt+At, size=EMBEDDING_DIM, window=5, min_count=1, workers=4)

def get_embedding(word):
    if word in w2vec:
        return w2vec[word]
    else:
        return np.zeros((1,EMBEDDING_DIM))

#Готовим матрицу коэффициентов
embedding_matrix = np.zeros((len(vocab), EMBEDDING_DIM))
for word, i in vocab.items():
    embedding_vector = np.zeros((1,EMBEDDING_DIM))
    embedding_vector = get_embedding(word)
    embedding_matrix[i] = embedding_vector 

#Готовый слой Керас
embedding_layer_X = L.Embedding(len(vocab),
                            EMBEDDING_DIM,
                            weights=[embedding_matrix], 
                            trainable=False, 
                            mask_zero=True)
embedding_layer_D = L.Embedding(len(vocab),
                            EMBEDDING_DIM,
                            weights=[embedding_matrix], 
                            trainable=False, 
                            mask_zero=True)

embedding_matrix.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


(25000, 64)

In [ ]:
MAX_LEN = 1000
N_LSTM = 256
N_EMBED = 64

N_VOCAB = len(vocab)


def build_model():
    X = L.Input(shape=(None,))
    D = L.Input(shape=(None,))
    e = embedding_layer_X(X)
    d = embedding_layer_D(D)

    enc, fh, fc, bh, bc = L.Bidirectional(L.LSTM(units=N_LSTM, return_sequences=True, return_state=True, dropout=0.1))(e)   
    h = L.Concatenate()([fh, bh])
    c = L.Concatenate()([fc, bc])

    #decoder
    dec = L.LSTM(N_LSTM*2, return_sequences=True, dropout=0.1)(d, initial_state=[h, c])
    
    #attention
    att = L.Attention(use_scale=True)([dec, enc])                                      
    out = L.Concatenate()([dec, att]) 

    #d = L.TimeDistributed(L.Dense(N_LSTM, activation='relu'))(out) # Убрал боттлнэк
    d = L.Dropout(0.2)(out)
    Y = L.TimeDistributed(L.Dense(N_VOCAB, activation='softmax'))(d)
    return keras.models.Model(inputs=[X,D], outputs=Y)
    
model = build_model()
opt = keras.optimizers.Adam(lr=0.0001)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, None, 64)     1600000     input_9[0][0]                    
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
bidirectional_4 (Bidirectional) [(None, None, 512),  657408      embedding_7[0][0]                
_______________________________________________________________________________________

In [ ]:
BATCH_SIZE = 32

def to_text(seq):
    return " ".join([vocab_inverse[idx] for idx in seq])

def to_sequence(text, vocab=vocab):
    tokens = text
    return [vocab[w] if w in vocab else vocab[UNK] for w in tokens]

def to_matrix(texts, maxlen=0):
    seqs = [to_sequence(text) for text in texts]
    if maxlen == 0:
        maxlen = min(MAX_LEN, max(list(map(len, seqs))))
    return keras.preprocessing.sequence.pad_sequences(seqs, maxlen=maxlen, dtype='int32', padding='post', truncating='post', value=0)

import random

def shuffleXY(x, y):
    Z = list(zip(x, y))
    random.shuffle(Z)
    return zip(*Z)

def train_gen(x, y):
    offset = 0
    count = BATCH_SIZE
    while True:
        xt = to_matrix(x[offset:offset+count], 0)
        yt = to_matrix(y[offset:offset+count], 0)
        dt = np.zeros_like(yt)
        dt[:, 1:] = yt[:,:-1]
        dt[:, 0] = 1 # START
        yield [xt, dt], yt
        offset += count
        if offset >= len(x)//BATCH_SIZE*BATCH_SIZE:
            offset = 0
            
def _schedule(epoch, lr):
    if epoch < 3:
        return 0.001
    else:
        return lr * tf.math.exp(-0.1)

def train_model(initial_epoch, n_epochs, train_gen):
    model.fit(train_gen, epochs=n_epochs,
                steps_per_epoch=len(Qt)//BATCH_SIZE,  initial_epoch=initial_epoch, 
                callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath='./model.{epoch:03d}.hdf5')
])
    
len(Qt), len(At)

(137616, 137616)

In [ ]:
opt = keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
train_model(0, 20, train_gen(Qt, At))

Epoch 1/20
4300/4300 [==============================] - 274s 64ms/step - loss: 1.5810 - accuracy: 0.2027
Epoch 2/20
4300/4300 [==============================] - 274s 64ms/step - loss: 1.4130 - accuracy: 0.2419
Epoch 3/20
4300/4300 [==============================] - 272s 63ms/step - loss: 1.3667 - accuracy: 0.2502
Epoch 4/20
4300/4300 [==============================] - 272s 63ms/step - loss: 1.3361 - accuracy: 0.2552
Epoch 5/20
4300/4300 [==============================] - 272s 63ms/step - loss: 1.3117 - accuracy: 0.2588
Epoch 6/20
4300/4300 [==============================] - 273s 63ms/step - loss: 1.2908 - accuracy: 0.2613
Epoch 7/20
4300/4300 [==============================] - 272s 63ms/step - loss: 1.2721 - accuracy: 0.2638
Epoch 8/20
4300/4300 [==============================] - 272s 63ms/step - loss: 1.2552 - accuracy: 0.2655
Epoch 9/20
4300/4300 [==============================] - 272s 63ms/step - loss: 1.2396 - accuracy: 0.2670
Epoch 10/20
4300/4300 [==============================] 

In [ ]:
def to_sequence(line):
    return [vocab[c] if c in vocab else vocab[PAD] for c in line]

def to_text(seq):
    return ' '.join([vocab_inverse[idx] if idx in vocab_inverse else UNK for idx in seq])

def mend_proba(p):
    p = np.log(p)
    p[0] = -100
    p = np.exp(p)/np.sum(np.exp(p))
    return p

def sample(proba, temp=1.0, mend=False, method='choice'):
    if mend:
        proba = mend_proba(proba)
    if temp != 1.0:
        proba = np.log(proba) / temp
        proba = np.exp(proba)/np.sum(np.exp(proba))    
    if method == 'choice':
        return np.random.choice(N_VOCAB, p=proba)
    elif method == 'multinomial':
        proba *= 0.9999
        return np.argmax(np.random.multinomial(1, proba, 1))

MAX_LEN = 50

def get_generation(seed="", temp=1.0, mend=False, method='choice'):
    seq = [1] # <START>
    x = to_matrix([tokenize(seed) + [END]], maxlen=0)
    #print(x)
    idx = 0
    while len(seq) < MAX_LEN:
        d = np.array([seq])
        #print(d)
        y = model.predict([x,d])
        p = y[0][-1]
        idx = sample(p, temp=temp, mend=mend, method=method)
        seq.append(idx)
        if idx in [0,2]: #<PAD><END>
            break
    return to_text(seq[1:-1])

model.reset_states()

seed = "Hello! How are you?"

print("Dialog question: ", seed)
print("Dialog possible answers:")

for i in range(10):
    print(str(i+1)+'.', get_generation(seed))

Dialog question:  Hello! How are you?
Dialog possible answers:
1. guess i could how s cease taking things i m going to come over to ignore and shoes
2. oh thank you
3. zip god
4. oh well then i m what i need
5. neil and i m just jon woman what are you doing
6. bad i m here
7. i m flattered
8. you better start right out of the bushes van
9. <UNK> mall we all gotta talk fast none of our neighbor look at the wheel n being laughing an old little girl of <UNK> look at your ass around in the building read it bothered
10. oh wh what was that that girl
